# Test for å lage kontroller av Finansiering av investeringene

## Krav for at tabellen skal være korrekt
1. Andel av brutto investeringsutgifter skal være 100 % per kommune per årgang
2. Beløp summert per kommune per årgang til tilsvare brutto investeringsutgifter
3. Årlig utvikling skal være fornuftig 
    - Definisjon av fornuftig utvikling må bestemmes

### Steg
- Importerer tabeller fra Linux. Trenger to tabeller:
    - Finansiering av investeringene
    - Finansielle grunnlagsdata 
    
1. Kontroller om andel er 100 %
    - Ok med avvik på 1 %
    - Hvor mange kommuner har høyere avvik
        - Er dette mer/mindre enn tidligere årganger
        - Ta ut liste over kommuner med avvik
2. Kontroller om sum av investeringene tilsvarer brutto investeringsutgifter for kommunen
    - Hvor mange avviker
    - Hvor betydelig er avviket
    - Er det sammenheng mellom kommunene som avviker på kontroll 1 og 2
3. Kontroller den årlige utviklingen for alle tilgjengelige årganger
    - Prosentvis utvikling 
    - Plotte utviklingen
    - Se om HP-metoden fra Kostra biblioteket kan være et alternativ



In [ ]:
import pandas as pd
import numpy as np
import glob

## Importerer og ordner data

In [ ]:
#Sjekker hva som ligger i mappen på linux
#Har tidligere lagret test-data på egen Linux mappe
paths = glob.glob("/ssb/bruker/mtk/Testdata/*")
paths

In [ ]:
#Henter filen, sjekker innhold
fininv = pd.read_csv("/ssb/bruker/mtk/Testdata/finansiering_av_investeringene.csv", sep = ';')
fininv.info()
fininv.head(20)

In [ ]:
fingr = pd.read_csv('/ssb/bruker/mtk/Testdata/finansielle_grunnlagsdata.csv', sep = ';')
fingr.dtypes
fingr.info()
fingr.head()

In [ ]:
#Retter datatyde
#fininv['andelinv'] = fininv['andelinv'].astype("float64")
#fininv.convert_dtypes().info()

## Kontroll 1: Sjekker andel av brutto investeringsutgifter

In [ ]:
#Filtrerer ut Halden og AGI23
#AGI23 = fininv[((fininv['kommuneregion'] == '3001')  | (fininv['kommuneregion'] == '0101')) & (fininv['art_kap'] == 'AGI23')]
#AGI23.head(10)

In [ ]:
#Grupperer finansiering av investeringene etter kommuneregion og periode
gruppert_alle = fininv.groupby(['kommuneregion', 'periode'])[['belop','andelinv']].sum()
gruppert_alle

In [ ]:
#Filtrerer ut observasjoner hvor andelinv avviker fra 100 med intervall på 0.5
diff = gruppert_alle[(gruppert_alle['andelinv'] >= 100.5) | (gruppert_alle['andelinv'] <= 99.5)]
diff.head(20)

In [ ]:
kontroll=diff.drop(columns=['belop'])
kontroll
juks=fingr.drop(columns=['art_kap', 'belop'])
juks=juks.drop_duplicates(subset=['periode','kommuneregion'], keep= 'first')
juks
#En merge ser ut til å løse groupby problemet. Det er riktignok juks... 
test=pd.merge(kontroll, juks, on=['periode','kommuneregion'])
test

### De to neste stegene grupperer, filtrerer og viser antall for en årgang. 
Mål: Hvordan iterere over alle årganger?

In [ ]:
#Filtrerer ut 2021
p2021 = fininv[(fininv['periode'] == 2021)]
#Grupperer finansiering av investeringene etter kommuneregion og periode
gruppert_2021 = p2021.groupby(['kommuneregion', 'periode'])[['belop','andelinv']].sum()
#Filtrerer ut observasjoner hvor andelinv avviker fra 100 med intervall på 1.0
diff2021 = gruppert_2021[(gruppert_2021['andelinv'] >= 101.0) | (gruppert_2021['andelinv'] <= 99.0)]

#2020
p2020 = fininv[(fininv['periode'] == 2020)]
gruppert_2020 = p2020.groupby(['kommuneregion', 'periode'])[['belop','andelinv']].sum()
diff2020= gruppert_2020[(gruppert_2020['andelinv'] >= 101.0) | (gruppert_2020['andelinv'] <= 99.0)]

print('Antall kommuner som i 2021 hadde avvik på mer enn 1 %:', len(diff2021))
print('Antall kommuner som i 2020 hadde avvik på mer enn 1 %:', len(diff2020))

In [ ]:
p2020_test=fininv.loc[(fininv['periode']==2020) & (fininv['kommuneregion']=='0301'),'andelinv'].sum()
p2020_test

## Kontroll 2: Sjekker om sum beløp stemmer overens med brutto investeringutgifter

In [ ]:
gruppert_alle.info
#fingr.head()



In [ ]:
biu = fingr[(fingr['art_kap']=='AGI1')]
#biu=biu.rename(columns={'belop':'biu'})
biu


In [ ]:
#biu.merge(gruppert_alle, left_on=['periode', 'kommuneregion'], right_on=['periode','kommuneregion'])
merged=biu.merge(gruppert_alle, on=['periode', 'kommuneregion'])
merged=merged.rename(columns={'belop_x':'biu','belop_y':'summert'})
merged=merged.drop(columns=['andelinv'])
merged